<br />

<div style="text-align: center;">
<font size="7">Keras simple CNN</font>
<br /> 

    
</div>
<br />


<div style="text-align: right;">
<font size="4">2020/11/11</font>
<br />
<font size="4">Ryutaro Hashimoto</font>
</div>

___

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Launching-a-Sagemaker-session" data-toc-modified-id="Launching-a-Sagemaker-session-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Launching a Sagemaker session</a></span></li><li><span><a href="#Prepare-the-dataset-for-training" data-toc-modified-id="Prepare-the-dataset-for-training-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prepare the dataset for training</a></span></li></ul></li><li><span><a href="#Train-the-model" data-toc-modified-id="Train-the-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Train the model</a></span><ul class="toc-item"><li><span><a href="#Specifying-the-Instance-Type" data-toc-modified-id="Specifying-the-Instance-Type-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Specifying the Instance Type</a></span></li><li><span><a href="#Setting-for-hyperparameters" data-toc-modified-id="Setting-for-hyperparameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setting for hyperparameters</a></span></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Metrics</a></span></li><li><span><a href="#Tags" data-toc-modified-id="Tags-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Tags</a></span></li><li><span><a href="#Setting-for-estimator" data-toc-modified-id="Setting-for-estimator-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setting for estimator</a></span></li><li><span><a href="#Specify-data-input-and-output" data-toc-modified-id="Specify-data-input-and-output-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Specify data input and output</a></span></li><li><span><a href="#Execute-Training" data-toc-modified-id="Execute-Training-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Execute Training</a></span></li><li><span><a href="#Checking-the-accuracy-of-a-model-with-TensorBoard" data-toc-modified-id="Checking-the-accuracy-of-a-model-with-TensorBoard-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Checking the accuracy of a model with TensorBoard</a></span></li></ul></li><li><span><a href="#Predict-by-trained-Model" data-toc-modified-id="Predict-by-trained-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Predict by trained Model</a></span><ul class="toc-item"><li><span><a href="#Deploy-the-trained-model" data-toc-modified-id="Deploy-the-trained-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Deploy the trained model</a></span></li><li><span><a href="#Invoke-the-endpoint" data-toc-modified-id="Invoke-the-endpoint-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Invoke the endpoint</a></span></li><li><span><a href="#Download-the-dataset-for-prediction" data-toc-modified-id="Download-the-dataset-for-prediction-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Download the dataset for prediction</a></span></li><li><span><a href="#Prediction" data-toc-modified-id="Prediction-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Prediction</a></span></li><li><span><a href="#Accuracy" data-toc-modified-id="Accuracy-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Accuracy</a></span></li><li><span><a href="#Confusion-Matrix" data-toc-modified-id="Confusion-Matrix-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Confusion Matrix</a></span></li></ul></li><li><span><a href="#Cleanup" data-toc-modified-id="Cleanup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cleanup</a></span></li></ul></div>

## Setup

### Launching a Sagemaker session

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

role = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx'    # ← your iam role ARN

### Prepare the dataset for training

Skip the next code since you have already downloaded it.

Next, we upload the data to Amazon S3:

In [3]:
from sagemaker.s3 import S3Uploader

bucket = 'sagemaker-tutorial-hashimoto'
dataset_uri = S3Uploader.upload('data', 's3://{}/tf-cifar10-example/data'.format(bucket))

display(dataset_uri)

's3://sagemaker-tutorial-hashimoto/tf-cifar10-example/data'

## Train the model

### Specifying the Instance Type

In [5]:
instance_type = 'ml.p2.xlarge'

### Setting for hyperparameters

In [6]:
hyperparameters = {'epochs': 10, 'batch-size': 256}

### Metrics

In [7]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': '.*loss: ([0-9\\.]+) - accuracy: [0-9\\.]+.*'},
    {'Name': 'train:accuracy', 'Regex': '.*loss: [0-9\\.]+ - accuracy: ([0-9\\.]+).*'},
    {'Name': 'validation:accuracy', 'Regex': '.*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: ([0-9\\.]+).*'},
    {'Name': 'validation:loss', 'Regex': '.*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_accuracy: [0-9\\.]+.*'},
    {'Name': 'sec/steps', 'Regex': '.* - \d+s (\d+)[mu]s/step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: [0-9\\.]+'}
]

### Tags

In [8]:
tags = [{'Key': 'Project', 'Value': 'cifar10'}, {'Key': 'TensorBoard', 'Value': 'file'}]

### Setting for estimator

In [9]:
import subprocess

from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='cifar10_keras_main.py',
                       source_dir='source_dir',
                       metric_definitions=metric_definitions,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='1.15.2',
                       py_version='py3',
                       instance_count=1,
                       instance_type=instance_type,
                       base_job_name='cifar10-tf',
                       tags=tags)

In [10]:
help(TensorFlow)

Help on class TensorFlow in module sagemaker.tensorflow.estimator:

class TensorFlow(sagemaker.estimator.Framework)
 |  TensorFlow(py_version=None, framework_version=None, model_dir=None, image_uri=None, distribution=None, **kwargs)
 |  
 |  Handle end-to-end training and deployment of user-provided TensorFlow code.
 |  
 |  Method resolution order:
 |      TensorFlow
 |      sagemaker.estimator.Framework
 |      sagemaker.estimator.EstimatorBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, py_version=None, framework_version=None, model_dir=None, image_uri=None, distribution=None, **kwargs)
 |      Initialize a ``TensorFlow`` estimator.
 |      
 |      Args:
 |          py_version (str): Python version you want to use for executing your model training
 |              code. Defaults to ``None``. Required unless ``image_uri`` is provided.
 |          framework_version (str): TensorFlow version you want to use for executing your model
 |              tra

### Specify data input and output

In [11]:
inputs = {
    'train': '{}/train'.format(dataset_uri),
    'validation': '{}/validation'.format(dataset_uri),
    'eval': '{}/eval'.format(dataset_uri),
}

### Execute Training

In [ ]:
estimator.fit(inputs)

2021-02-08 06:06:01 Starting - Starting the training job...
2021-02-08 06:06:25 Starting - Launching requested ML instancesProfilerReport-1612764359: InProgress
......
2021-02-08 06:07:32 Starting - Preparing the instances for training.........

### Checking the accuracy of a model with TensorBoard

Using the visualization tool [TensorBoard](https://www.tensorflow.org/tensorboard), we can compare our training jobs.

In a local setting, install TensorBoard with `pip install tensorboard`. Then run the command generated by the following code:

In [ ]:
!python generate_tensorboard_command.py

In [ ]:
! AWS_REGION=us-west-2 tensorboard --logdir file:"s3://sagemaker-us-west-2-005242542034/cifar10-tf-2021-02-08-04-01-54-836/model"

After running that command, we can access TensorBoard locally at http://localhost:6006.

Based on the TensorBoard metrics, we can see that:
1. All jobs run for 10 epochs (0 - 9).
1. Both File Mode and Pipe Mode run for ~1 minute - Pipe Mode doesn't affect training performance.
1. Distributed training runs for only 45 seconds.
1. All of the training jobs resulted in similar validation accuracy.

This example uses a relatively small dataset (179 MB). For larger datasets, Pipe Mode can significantly reduce training time because it does not copy the entire dataset into local memory.

## Predict by trained Model

### Deploy the trained model

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Invoke the endpoint

I'll try to generate a random matrix and see if the predictor is working.

In [ ]:
import numpy as np

data = np.random.randn(1, 32, 32, 3)
print('Predicted class: {}'.format(np.argmax(predictor.predict(data)['predictions'])))

### Download the dataset for prediction

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### Prediction

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def predict(data):
    predictions = predictor.predict(data)['predictions']
    return predictions


predicted = []
actual = []
batches = 0
batch_size = 128

datagen = ImageDataGenerator()
for data in datagen.flow(x_test, y_test, batch_size=batch_size):
    for i, prediction in enumerate(predict(data[0])):
        predicted.append(np.argmax(prediction))
        actual.append(data[1][i][0])

    batches += 1
    if batches >= len(x_test) / batch_size:
        break

### Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_pred=predicted, y_true=actual)
display('Average accuracy: {}%'.format(round(accuracy * 100, 2)))

### Confusion Matrix

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_pred=predicted, y_true=actual)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sn.set(rc={'figure.figsize': (11.7,8.27)})
sn.set(font_scale=1.4)  # for label size
sn.heatmap(cm, annot=True, annot_kws={"size": 10})  # font size

## Cleanup

To avoid incurring extra charges to your AWS account, let's delete the endpoint we created:

In [ ]:
predictor.delete_endpoint()